In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [2]:
missing_query = f"""
SELECT
    * 
FROM
    (
    SELECT
        repo_info.repo_id,
        repo.repo_name,
repo.repo_git, 
        MAX ( pull_request_count ) AS max_pr_count,
        COUNT ( * ) AS meta_count 
    FROM
        repo_info,
        repo -- WHERE issues_enabled = 'true' 
    WHERE
        pull_request_count >= 1
        AND repo.repo_id = repo_info.repo_id 
    GROUP BY
        repo_info.repo_id,
        repo.repo_name, 
 repo.repo_git 
    ORDER BY
        repo_info.repo_id,
        repo.repo_name 
    ) yy
    LEFT OUTER JOIN (
    SELECT A
        .repo_id,
        A.repo_name,
        b.pull_request_count,
        d.repo_id AS pull_request_repo_id,
        e.last_collected,
        f.last_pr_collected,
        COUNT ( * ) AS pull_requests_collected,
        ( b.pull_request_count - COUNT ( * ) ) AS pull_requests_missing,
        ABS ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_abs,
        ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_prs 
    FROM
        augur_data.repo A,
        augur_data.pull_requests d,
        augur_data.repo_info b,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_collected FROM augur_data.repo_info GROUP BY repo_id ORDER BY repo_id ) e,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_pr_collected FROM augur_data.pull_requests GROUP BY repo_id ORDER BY repo_id ) f 
    WHERE
        A.repo_id = b.repo_id 
        AND LOWER ( A.repo_git ) LIKE'%github.com%' 
        AND A.repo_id = d.repo_id 
        AND b.repo_id = d.repo_id 
        AND e.repo_id = A.repo_id 
        AND b.data_collection_date = e.last_collected 
        AND f.repo_id = A.repo_id -- AND d.pull_request_id IS NULL
        
    GROUP BY
        A.repo_id,
        d.repo_id,
        b.pull_request_count,
        e.last_collected,
        f.last_pr_collected 
    ORDER BY
        ratio_abs desc
    ) zz ON yy.repo_id = zz.repo_id 
ORDER BY
    ratio_abs;
    """
missing_data = pd.read_sql_query(missing_query, con=engine)
print(missing_data)

      repo_id                repo_name  \
0       25638          start.spring.io   
1       30395                  builder   
2       25432          rabbitmq-server   
3       25433         rabbitmq-codegen   
4       25434             rabbitmq-bql   
...       ...                      ...   
1763    26373  concourse-pypi-resource   
1764    26439               ksm-sample   
1765    28024   terraform-provider-vmc   
1766    26440                 docs-ksm   
1767    27938  terraform-provider-vra7   

                                               repo_git  max_pr_count  \
0          https://github.com/spring-io/start.spring.io           236   
1                https://github.com/projectriff/builder           351   
2           https://github.com/rabbitmq/rabbitmq-server          1189   
3          https://github.com/rabbitmq/rabbitmq-codegen            12   
4              https://github.com/rabbitmq/rabbitmq-bql             2   
...                                                 ...  

In [3]:
df = missing_data.loc[missing_data['ratio_abs'] < 1]
#print(df)
df1 = df[['repo_id', 'repo_git', 'last_pr_collected', 'pull_requests_missing']]
print(df1)
#df1.to_csv('/Users/dafoster/gitrepos/augur-pivotal/code/health_scripts/output/missing.csv')

   repo_id  repo_id                                      repo_git  \
0    25638    25638  https://github.com/spring-io/start.spring.io   
1    30395    30395        https://github.com/projectriff/builder   

    last_pr_collected  pull_requests_missing  
0 2020-09-10 16:59:19                      3  
1 2020-09-10 09:15:13                      1  
